In [1]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [2]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [3]:
NL1 = 2
NL2 = 2
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

In [4]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [5]:
## Let us compute the thermal density matrix
beta1 = 1  #left baths
beta2 = 0.6 #right baths

rho_th = (-beta1*H_S).expm()/((-beta1*H_S).expm()).tr() #left thermal density matrix
print(rho_th)

Quantum object: dims=[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape=(64, 64), type='oper', dtype=Dense, isherm=True
Qobj data =
[[3.80777093e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.03175783e-03 3.29634248e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.29634248e-06 1.02846676e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.61523826e-02
  1.79974202e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.79974202e-04
  5.63320690e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.53616443e-01]]


In [6]:
## Now we set parameters for the  non-eq setup

beta_list_left = np.linspace(beta1,beta1,NL1)
beta_list_right = np.linspace(beta2,beta2,NL2)

mu_list_left = np.linspace(-0.5,-0.5,NL1)
mu_list_right = np.linspace(-0.5,-0.5,NL2)

gamma_list_left = np.linspace(1,1,NL1)  #coupling strengths to left baths
gamma_list_right = np.linspace(1,1,NL2)  #coupling strengths to right baths

tb = 0.01

In [7]:
number = len(eigenergies)

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL1):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*C(k,y,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*D(y,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

            for l in range(NL2):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*C(k,y,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*D(y,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


KeyboardInterrupt: 

In [8]:
print(number)  #should be 2^N

64


In [9]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[(0.1536164432357538+0j), (0.0565123312923138+0j), (0.056463896272977196+0j), (0.05633178086692868+0j), (0.05615180727947185+0j), (0.055972408687012254+0j), (0.055841443273902135+0j), (0.020789724855111803+0j), (0.020771906607265127+0j), (0.020751657685019565+0j), (0.02072330406551786+0j), (0.020710947145894693+0j), (0.020674601227246823+0j), (0.02065709548273864+0j), (0.020629746935722008+0j), (0.02059109842879765+0j), (0.020560107729941855+0j), (0.02054291894580991+0j), (0.020509188788367938+0j), (0.02049063479151697+0j), (0.020437296949901474+0j), (0.020361504768194885+0j), (0.007648112361806578+0j), (0.007641557394746083+0j), (0.007634108232546067+0j), (0.007625380146612352+0j), (0.007623677518848587+0j), (0.00761913166216302+0j), (0.00760576074592198+0j), (0.007599320742415012+0j), (0.007589259774221686+0j), (0.007582021888507876+0j), (0.0075750417830922415+0j), (0.007563640942115659+0j), (0.007557317541814783+0j), (0.007550840392263523+0j), (0.00754490891034441+0j), (0.0075380832

In [10]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [11]:
print(A)

[[ 1.13574613e+000+0.00000000e+00j -8.49605930e-001+0.00000000e+00j
  -1.21821923e+000+0.00000000e+00j ...  0.00000000e+000+0.00000000e+00j
  -5.78272338e-126+0.00000000e+00j  0.00000000e+000+0.00000000e+00j]
 [-1.89572707e-001+1.05032085e-19j  1.79634272e+000-3.47469856e-17j
   7.98654563e-018+5.54758123e-17j ... -9.15328014e-033-9.24785823e-49j
  -5.87261726e-033+9.83285569e-49j -2.54914290e-033-1.20289214e-49j]
 [-2.71588481e-001-7.45388994e-20j  2.21412015e-017-1.39675733e-17j
   2.08294089e+000+8.33753673e-17j ... -1.26222269e-032+1.89375879e-49j
  -1.59624176e-033-7.31079584e-50j -1.05660838e-032+1.08253512e-49j]
 ...
 [ 0.00000000e+000+0.00000000e+00j  4.09500135e-034-2.67997059e-51j
  -1.36610799e-034-2.75883851e-50j ...  4.26903572e+000-4.91956736e-17j
  -4.70383093e-017+1.43446739e-17j -1.05493590e+000-1.39645240e-16j]
 [-1.37969086e-081-5.13942435e-99j -2.74175204e-034+7.61470398e-51j
  -1.31682022e-035+8.06325552e-50j ... -4.04473985e-018+4.04119419e-17j
   4.71646686e+000+

In [12]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

print(np.linalg.matrix_rank(A))





63
[[-2.45166743e+00+0.00000000e+00j  7.10328766e-01+1.05018262e-17j
   1.16081577e+00+4.22212865e-17j ... -1.47338940e-33-2.27258422e-50j
  -7.26750648e-34+5.01400042e-50j -3.22229367e-35+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -2.55745996e+00+0.00000000e+00j
  -9.34295971e-03+8.54342675e-17j ... -2.24927461e-32-4.14293119e-48j
  -3.16371709e-32+2.28546459e-48j -2.08770243e-33-5.94038004e-49j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -2.49585810e+00+0.00000000e+00j ... -4.95831686e-32+1.64233917e-48j
  -6.06678882e-32-1.86873995e-48j  1.86651205e-32+4.63808124e-49j]
 ...
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ... -3.70374477e+00+0.00000000e+00j
   1.22831339e+00+8.52989986e-17j  2.47543137e+00-1.76837328e-16j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
  -2.82590708e+00+0.00000000e+00j  2.8259070

In [13]:
print(diag_r)

[(-2.451667426237998+0j), (-2.5574599563573286+0j), (-2.495858099431737+0j), (-2.249381061730985+0j), (-2.1646880224904206+0j), (-2.1943571318223394+0j), (-2.3736299242305035+0j), (-2.691577315895321+0j), (-2.666392155080533+0j), (-2.378334061776155+0j), (-2.579916670122432+0j), (-2.481580957218562+0j), (-2.34340529290274+0j), (-2.5630865864462296+0j), (-2.4880766149850246+0j), (-2.584504525581906+0j), (-2.4883483436279086+0j), (-2.6211411530385647+0j), (-2.3823105707693504+0j), (-2.3369756072688066+0j), (-2.37200535606612+0j), (-2.438450247997556+0j), (-2.8063794510117024+0j), (-3.0273226381853227+0j), (-2.987645904220277+0j), (-2.951734755977438+0j), (-2.9115578233530206+0j), (-3.029798091732018+0j), (-2.93940134772522+0j), (-2.999932129886+0j), (-3.020953203834045+0j), (-2.9769084394448466+0j), (-3.004484747461636+0j), (-3.0390126492444747+0j), (-2.9053979300028527+0j), (-3.004961440745749+0j), (-3.017992986470041+0j), (-3.0338780402869934+0j), (-2.9760738668685796+0j), (-3.02295692

In [14]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
print(A_new)

64
(64, 64)
[[ 1.13574613e+000+0.00000000e+00j -8.49605930e-001+0.00000000e+00j
  -1.21821923e+000+0.00000000e+00j ...  0.00000000e+000+0.00000000e+00j
  -5.78272338e-126+0.00000000e+00j  0.00000000e+000+0.00000000e+00j]
 [-1.89572707e-001+1.05032085e-19j  1.79634272e+000-3.47469856e-17j
   7.98654563e-018+5.54758123e-17j ... -9.15328014e-033-9.24785823e-49j
  -5.87261726e-033+9.83285569e-49j -2.54914290e-033-1.20289214e-49j]
 [-2.71588481e-001-7.45388994e-20j  2.21412015e-017-1.39675733e-17j
   2.08294089e+000+8.33753673e-17j ... -1.26222269e-032+1.89375879e-49j
  -1.59624176e-033-7.31079584e-50j -1.05660838e-032+1.08253512e-49j]
 ...
 [ 0.00000000e+000+0.00000000e+00j  4.09500135e-034-2.67997059e-51j
  -1.36610799e-034-2.75883851e-50j ...  4.26903572e+000-4.91956736e-17j
  -4.70383093e-017+1.43446739e-17j -1.05493590e+000-1.39645240e-16j]
 [-1.37969086e-081-5.13942435e-99j -2.74175204e-034+7.61470398e-51j
  -1.31682022e-035+8.06325552e-50j ... -4.04473985e-018+4.04119419e-17j
   4.71

In [15]:
print(A.shape)

(64, 64)


In [16]:
print(np.linalg.matrix_rank(A))
print(A.shape)

63
(64, 64)


In [17]:
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(x)

[3.00075127e-01-8.61196434e-18j 6.69558112e-02-4.43431313e-19j
 6.68984254e-02-4.19858488e-18j 6.67418950e-02-3.37272604e-18j
 6.65286623e-02-3.22045838e-18j 6.63161109e-02+1.36705655e-18j
 6.61609430e-02-1.74762581e-18j 1.49398609e-02+1.21578622e-18j
 1.49270564e-02+1.49704999e-19j 1.49125051e-02-6.10551957e-20j
 1.48921297e-02-1.15025120e-19j 1.48832498e-02-1.94193162e-19j
 1.48571310e-02-4.07161129e-19j 1.48445511e-02+2.03709852e-19j
 1.48248979e-02+4.39535597e-19j 1.47971244e-02+4.32316354e-19j
 1.47748540e-02+6.28530575e-19j 1.47625018e-02-2.91444314e-19j
 1.47382627e-02+5.62719781e-19j 1.47249295e-02+4.10969763e-19j
 1.46866000e-02-4.75904215e-19j 1.46321344e-02-8.57514210e-19j
 3.33353355e-03+4.01771844e-19j 3.33067648e-03+3.57547327e-19j
 3.32742966e-03+3.62065489e-19j 3.32362540e-03+3.32402740e-19j
 3.32288329e-03+3.06278289e-19j 3.32090192e-03+3.46767557e-19j
 3.31507402e-03+7.05919138e-20j 3.31226706e-03+3.22674540e-19j
 3.30788185e-03+2.83059435e-19j 3.30472712e-03+1.771905

In [18]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[-8.32667268e-17+3.85185989e-34j -5.55111512e-17+7.70371978e-33j
 -5.55111512e-17+1.92592994e-33j  2.77555756e-17-3.08148791e-33j
  7.63278329e-17-1.05926147e-33j -1.38777878e-17-7.70371978e-34j
  2.08166817e-17-3.85185989e-33j  1.90819582e-17-7.70371978e-34j
  2.08166817e-17-9.62964972e-34j  3.46944695e-18+7.70371978e-34j
 -2.08166817e-17-7.70371978e-34j  6.93889390e-18+4.81482486e-34j
  1.73472348e-18-8.66668475e-34j  3.46944695e-18-7.46297853e-34j
 -1.38777878e-17+3.85185989e-34j -5.20417043e-18-3.85185989e-34j
  3.46944695e-18+9.62964972e-35j  1.38777878e-17+0.00000000e+00j
  1.73472348e-18-3.12963616e-34j  1.04083409e-17-3.85185989e-34j
 -2.25514052e-17+1.30000271e-33j -1.73472348e-17-1.44444746e-34j
  6.07153217e-18+3.85185989e-34j -6.93889390e-18+4.81482486e-34j
  0.00000000e+00-3.85185989e-34j -1.73472348e-18+9.62964972e-35j
 -1.73472348e-18+0.00000000e+00j  6.93889390e-18+0.00000000e+00j
 -2.60208521e-18-1.08333559e-34j -8.67361738e-19+2.88889492e-34j
 -3.46944695e-18-3.851859

In [19]:
#We have to tranfer the basis of the solution rho matrix to computational basis from the eigenbasis

x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

check = 1

print(U[:,check])
print(eigstates[check])

[ 0.00000000e+00+0.j -1.03425420e-84+0.j -1.29385200e-81+0.j
 -6.60377089e-23+0.j -2.02811560e-75+0.j -3.94016465e-21+0.j
  4.36262361e-17+0.j  3.93220192e-19+0.j  8.66497715e-18+0.j
 -4.17622434e-17+0.j -8.15604823e-18+0.j  1.18468672e-17+0.j
  8.03666527e-19+0.j  6.35123446e-17+0.j  1.14703171e-16+0.j
 -7.33868575e-17+0.j  1.55225137e-17+0.j -3.76611699e-17+0.j
  5.35043640e-18+0.j -4.91511623e-17+0.j -1.97885547e-17+0.j
  2.72396445e-17+0.j -1.95005694e-17+0.j  4.56317247e-17+0.j
 -3.44748267e-17+0.j -3.34075540e-18+0.j  1.31588542e-16+0.j
  3.96918990e-16+0.j  5.67913017e-17+0.j -3.41149140e-16+0.j
 -2.91107809e-16+0.j  4.08248290e-01+0.j  1.58368328e-17+0.j
  0.00000000e+00+0.j  5.55111512e-17+0.j -6.93889390e-17+0.j
  2.77555756e-17+0.j  6.93889390e-18+0.j  5.55111512e-17+0.j
 -4.44089210e-16+0.j  4.16333634e-17+0.j -1.94289029e-16+0.j
 -4.16333634e-17+0.j  4.66423775e-16+0.j  8.15320034e-17+0.j
  7.12134480e-16+0.j -7.20276902e-16+0.j  4.08248290e-01+0.j
  1.35525272e-17+0.j  1.

In [25]:
rho_comp = np.dot(np.conjugate(U.T),np.dot(rho,U))

print(rho_comp)
print(np.trace(rho_comp))

[[ 3.70322127e-05+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  8.78239546e-04+0.j  8.45452379e-04+0.j ...
   7.99788651e-20+0.j -1.36839818e-20+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  8.45452379e-04+0.j  1.24948794e-03+0.j ...
   2.77546727e-19+0.j -7.17285327e-20+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  7.99788651e-20+0.j  2.77546727e-19+0.j ...
   3.98671943e-02+0.j  2.33386809e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.36839818e-20+0.j -7.17285327e-20+0.j ...
   2.33386809e-02+0.j  4.05140732e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  3.00075127e-01+0.j]]
(1.0000000000000018+0j)


In [21]:
def L2_red(rho,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL1):
                op = commutator(rho*proj_i*create_sm_list_left[l]*proj_k,create_sm_list_left[l].dag())*C(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies) + commutator(create_sm_list_left[l].dag(),proj_i*create_sm_list_left[l]*proj_k*rho)*D(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += op
                sum += op.dag()

            for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*C(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies) + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*D(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [22]:
L2_redfield = L2_red(rho_comp,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb)

In [23]:
print(L2_redfield)

[[-2.61400523e-03+0.00000000e+00j -9.66804988e-04-1.19531035e-07j
   2.93275475e-04-6.43389149e-07j ... -4.39988869e-25-6.69888200e-23j
  -7.20745962e-26-4.97273622e-22j  0.00000000e+00+0.00000000e+00j]
 [-9.66804988e-04+1.19531035e-07j  7.90764205e-04+0.00000000e+00j
   3.30224216e-03-7.64373041e-07j ...  2.60803136e-19-2.55417756e-19j
  -4.14005116e-20+4.81647787e-20j -4.77279084e-30+4.77961183e-26j]
 [ 2.93275475e-04+6.43389149e-07j  3.30224216e-03+7.64373041e-07j
   2.36545737e-03+0.00000000e+00j ...  8.76832506e-19-9.32539784e-19j
  -2.22756759e-19+2.47495247e-19j  2.41864155e-28+5.28166807e-26j]
 ...
 [-4.39988869e-25+6.69888200e-23j  2.60803136e-19+2.55417756e-19j
   8.76832506e-19+9.32539784e-19j ... -7.63787292e-02+0.00000000e+00j
   2.81791270e-02+9.55154968e-06j -7.28303109e-03+1.57785785e-06j]
 [-7.20745962e-26+4.97273622e-22j -4.14005116e-20-4.81647787e-20j
  -2.22756759e-19-2.47495247e-19j ...  2.81791270e-02-9.55154968e-06j
  -9.30386304e-02+0.00000000e+00j -2.59961677e-

In [24]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
#We will also pass in L2_redfield operator

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

print(rho_th_check)

data_dict = {"dm_ness":rho_comp, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(F'../matlab/ness_data_NL1 = {NL1}_NL2 = {NL2}_NM = {NM}_3.mat',data_dict)


[[3.80777093e-04+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 1.03175783e-03+0.j 3.29634248e-06+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 3.29634248e-06+0.j 1.02846676e-03+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 ...
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  5.61523826e-02+0.j 1.79974202e-04+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  1.79974202e-04+0.j 5.63320690e-02+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 1.53616443e-01+0.j]]
